In [1]:
import json
import yaml
import zipfile

In [2]:
with open('data/data.json') as f:
    data = json.load(f)

In [3]:
dic = data[0]

In [4]:
magnetron = {
        'name' : dic['source_' + '1' + '_material'],
        'material' : {
            'name' : dic['source_' + '1' + '_material'],
            'lab_id' : None
        }}

In [5]:
def getPoweredAxes(arr):
    axes = []
    for i in range(len(arr)):
        if arr[i]:
            axes.append(str(i+1))
    return axes

In [6]:
def getYamlMagnetrons(dic, power_axes):
    axes = getPoweredAxes(power_axes)
    return [getYamlMagnetron(dic, axis) for axis in axes]

In [7]:
def getYamlMagnetron(dic, axis):
    magnetron = {
        'name' : dic['source_' + axis + '_material'],
        'material' : {
            'name' : dic['source_' + axis + '_material'],
            'lab_id' : None
        },
        'vapor_source' : {
            'setpoints' : {
            'set_power' : dic['source_' + axis + '_set_power_[W]'],
            'set_voltage' : None,
            'set_current' : None
            },
            'power' : {
                'values' : dic['source_' + axis + '_act_power_[W]'],
                'times' : dic['time_[s]'],
                'mean' : dic['source_' + axis + '_act_power_[W]_mean'],
                'error' : dic['source_' + axis + '_act_power_[W]_mean']
            },
            'voltage' : {
                'values' : dic['source_' + axis + '_voltage_[V]'],
                'times' : dic['time_[s]'],
                'mean' : dic['source_' + axis + '_voltage_[V]_mean'],
                'error' : dic['source_' + axis + '_voltage_[V]_std'],
            },
            'current' : {
                'values' : [],
                'times' : dic['time_[s]'],
                'mean' : None,
                'error' : None
            },
            'power_supply' : {
                'instrument_id' : dic['source_' + axis + '_supply'],
                'frequency' : None
            }
        },
        'position' : {
            'center_xyz' : None,
            'center_normal' : None,
            'rotation' : None
        }
    }
    return magnetron

# Environment

In [8]:
def getYamlQCM(dictionary, i):
    prefix = 'qcm_' + str(i) + '_'
    yamlQCM = {
            'crystal_info' : {
                'info' : None,
                'resonant_frequency' : None
            },
            'sensor_data' : {
                'values' : dictionary[prefix + 'frequency_[Hz]'],
                'mean' : dictionary[prefix + 'frequency_[Hz]_mean'],
                'error' : dictionary[prefix + 'frequency_rate_[/s2]_error'],
                'slope' : dictionary[prefix + 'frequency_rate_[/s2]'],
            },
            'remaining_lifetime' : {
                'value' : dictionary[prefix + 'lifetime_[%]']
            },
            'mass_deposition_rate' : {
                'value' : dictionary[prefix + 'mass_rate_[g/cm2s]'],
                'error' : dictionary[prefix + 'mass_rate_[g/cm2s]_error']
            }
        }
    return yamlQCM

In [9]:
def getYamlEnv(dictionary):
    environment = {
        'gas_flow' : {
            'values' :  dictionary['flow_[sccm]'],
            'mean' : dictionary['flow_[sccm]_mean'],
            'error' : dictionary['flow_[sccm]_std'],
            'gas' : 'Here we do the pubchem thing'
        },
        'setpoints' : {
            'pressure' : dictionary['set_pressure_[mTorr]'],
            'flow' : None,
        },
        'pressure' : {
            'values' : dictionary['act_pressure_[mTorr]'],
            'mean' : dictionary['act_pressure_[mTorr]_mean'],
            'error' : dictionary['act_pressure_[mTorr]_std']
        },
        'sensors' : [getYamlQCM(dictionary, i+1) for i in range(3)] if dic['qcms_active'] else []
    }
    return environment
    


In [10]:
#environment = getYamlEnv(dic)

In [11]:
#environment

In [12]:
data = {'data' : {
            'm_def' : "../uploads/1WE7CfSoQbS8LidbT0XLfw/raw/automate-solar_schema.archive.yaml#/definitions/section_definitions/AutomateSolarSputterDeposition",
            'quantity' : None,
            'name' : dic['series_id'],
            'lab_id' : "Uppsala University Åutomate-Solar",
            'description' : f'{dic["campaign_description"]}, {dic["campaign_id"]}, {dic["series_description"]}',
            'steps' : [
                {
                    'sources' : getYamlMagnetrons(dic, [True,False,False,False,False,False]),
                    'environment' : getYamlEnv(dic)
                }
            ]
    }
}

In [13]:
data

{'data': {'m_def': '../uploads/1WE7CfSoQbS8LidbT0XLfw/raw/automate-solar_schema.archive.yaml#/definitions/section_definitions/AutomateSolarSputterDeposition',
  'quantity': None,
  'name': 'C007_S002',
  'lab_id': 'Uppsala University Åutomate-Solar',
  'description': 'Testing Boundary Finder, C007, testing save ok',
  'steps': [{'sources': [{'name': 'Zr',
      'material': {'name': 'Zr', 'lab_id': None},
      'vapor_source': {'setpoints': {'set_power': 21,
        'set_voltage': None,
        'set_current': None},
       'power': {'values': [12.6,
         12.6,
         13.801,
         13.801,
         13.801,
         13.801,
         13.801,
         13.801,
         13.801,
         13.801,
         14.685,
         14.685,
         14.685,
         14.685,
         14.685,
         14.175,
         14.175,
         14.175,
         14.175,
         14.175,
         14.175,
         12.618,
         12.618,
         12.618,
         12.618,
         12.618,
         13.641,
     

In [14]:
#print(yaml.dump(data, default_flow_style=False))

In [15]:
#with open('data/data.archive.yaml', 'w') as outfile:
    #yaml.dump(data, outfile, default_flow_style=False)

In [16]:
#with zipfile.ZipFile('data/data.zip', 'w') as zipped_f:
    #zipped_f.writestr("data/data.archive.yaml", yaml.dump(data, default_flow_style=False))

In [17]:
#test = yaml.dump(data, default_flow_style=False)